In [ ]:
!pip install -q transformers gradio beautifulsoup4 requests torch

In [ ]:
import os
import requests
import gradio as gr
import torch
from bs4 import BeautifulSoup
from transformers import pipeline

# Load Hugging Face models
try:
    model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    print(f"Loading model: {model_name}")
    generator = pipeline("text-generation", model=model_name, torch_dtype=torch.float16, device_map="auto")
    print("Text generation model loaded successfully.")
except Exception as e:
    print(f"Error loading text generation model: {e}")
    generator = None

try:
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")
    print("Translation model loaded successfully.")
except Exception as e:
    print(f"Error loading translation model: {e}")
    translator = None

# Web scraping utility
class Website:
    def __init__(self, url):
        self.url = url
        headers = {"User-Agent": "Mozilla/5.0"}
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            self.body = response.content
            soup = BeautifulSoup(self.body, "html.parser")
            self.title = soup.title.string if soup.title else "No title found"
            if soup.body:
                for tag in soup.body(["script", "style", "img", "input"]):
                    tag.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = ""
            print(f"Successfully scraped: {url}")
        except requests.RequestException as e:
            print(f"Error scraping {url}: {e}")
            self.title = "Error"
            self.text = ""

    def get_contents(self):
        return f"Title: {self.title}\nContent: {self.text[:1000]}..."

# Generate brochure text and translate it if needed
def generate_brochure(company_name, url):
    if not generator or not translator:
        return "Model loading failed. Please check logs."

    print(f"Generating brochure for: {company_name} from {url}")
    website = Website(url)
    if not website.text:
        return "Failed to extract website content."

    prompt = f"Generate a professional company brochure for {company_name} based on this content:\n{website.get_contents()}"
    try:
        result = generator(prompt, max_length=500, do_sample=True, top_k=50, top_p=0.95)[0]["generated_text"]
        print("Brochure generation successful.")
    except Exception as e:
        print(f"Error generating brochure: {e}")
        return "Error generating brochure."

    try:
        translation = translator(result)[0]["translation_text"]
        print("Translation successful.")
        return translation
    except Exception as e:
        print(f"Error translating text: {e}")
        return "Error in translation."

# Gradio Interface
def on_select(selected):
    if selected:
        return generate_brochure("Ikano", selected.strip())
    return ""

options = [
    "https://bank.ikano", "https://ikanobank.at", "https://ikanobank.dk", "https://ikanobank.fi",
    "https://ikanobank.de", "https://ikanobank.no", "https://ikanobank.pl", "https://ikanobank.se",
    "https://ikano.co.uk", "https://open.bank.ikano"
]

with gr.Blocks() as demo:
    dropdown = gr.Dropdown(choices=options, label="Select a Banking Portal")
    output = gr.Textbox()
    dropdown.change(on_select, inputs=[dropdown], outputs=[output])

demo.launch(share=True, debug=True)


Loading model: mistralai/Mistral-7B-Instruct-v0.1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Text generation model loaded successfully.


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


Translation model loaded successfully.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6442221932923c0e74.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
